In [4]:
import joblib
import pandas as pd

In [1]:
# ht_model = joblib.load('height_model.pkl')
# wt_model = joblib.load('width_model.pkl')
# dt_model = joblib.load('depth_model.pkl')
generic_model = joblib.load('generic_model.pkl')

NameError: name 'joblib' is not defined

In [5]:
def get_model(entity_name):
    model = None
    
    match entity_name:
        case "height":
            return ht_model
        case "width":
            return wt_model
        case "depth":
            return dt_model
        case default:
            return generic_model

In [6]:
def get_probability_from_model(columns, entity_name):
    columns = normalize_columns(columns)
    
    # Load the saved model
    model = get_model(entity_name)

    feature_names = ['x_min', 'y_min', 'height', 'width', 'zero', 'plus_90', 'minus_90']

    # Convert the list to a DataFrame with correct feature names
    features = pd.DataFrame([columns], columns=feature_names)

    predicted_proba = model.predict_proba(features)
    return predicted_proba[0]

In [7]:
def normalize_columns(columns):
    
    scaling_factors = {
     'x_min': {'mean': np.float64(695.447529150472), 'std': np.float64(494.83945389032766)},
     'y_min': {'mean': np.float64(873.8738200999445), 'std': np.float64(488.82084409116834)}, 
     'height': {'mean': np.float64(49.70016657412548), 'std': np.float64(52.56778724634215)},
     'width': {'mean': np.float64(255.19614103275958), 'std': np.float64(219.7430970874373)}
    }

    new_columns = {
        'x_min':(columns['x_min'] - scaling_factors['x_min']['mean'])/scaling_factors['x_min']['std'],
        'y_min':(columns['y_min'] - scaling_factors['y_min']['mean'])/scaling_factors['y_min']['std'],
        'height':(columns['height'] - scaling_factors['height']['mean'])/scaling_factors['height']['std'],
        'width':(columns['width'] - scaling_factors['width']['mean'])/scaling_factors['width']['std'],
        'zero':columns['zero'],
        'plus_90':columns['plus_90'],
        'minus_90':columns['minus_90']
    }

    return new_columns

In [8]:
def expand_orientation(orientation):

    if orientation[-2] == '_':
        # zero
        return 1,0,0
    elif orientation[-4] == 'g':
        # neg
        return 0,0,1
    else:
        # pos
        return 0,1,0

In [9]:
## format:

## metadata_value x_min y_min height width orientation
## 200mg           600    400    45    67     ocr_results_0 

In [10]:
def get_paddle_data(image_link, category_id, entity_name):
    
    #call whatever the fuck you want to and get me the data
    a = {
        "orientation":"ocr_results_neg_90",
        "metadata_value":"1023g",
        "x_min":"1708.0",
        "x_max":"2075.0",
        "y_min":"1930.0",
        "y_max":"2168.0",
        ""2400","1023.0 gram"]
    
    return pd.Data

In [11]:
def get_probabilities(image_link, category_id, entity_name):
    
    option_df = get_paddle_data(image_link, category_id, entity_name)
    options = []
    for index, row in option_df.iterrows():
        
        zero, plus_90, minus_90 = expand_orientation(rows['orientation'])
        
        word = row['metadata_value']
        columns = {
            'x_min':row['x_min'],
            'y_min':row['y_min'],
            'height':row['width'],
            'width':row['height'],
            'zero':zero,
            'plus_90':plus_90,
            'minus_90':minus_90
        }

        prob = get_probability_from_model(columns, entity_name)
        options.append({'word':word,'prob':prob[1]})
    return options

In [13]:
def predictor(image_link, category_id, entity_name):
    a = get_probabilities(image_link, category_id, entity_name)
    max_prob = -1
    best_guess = ""
    for values in a:
        if(values['prob']>max_prob):
            max_prob = values['prob']
            best_guess = values['word']
    print(best_guess)

In [14]:
predictor("https://m.media-amazon.com/images/I/11gHj8dhhrL.jpg",792578,"width")

AttributeError: 'int' object has no attribute 'iterrows'